# Yelp dataset cleaning

In [16]:
#Cargamos las librerías necesarias para hacer la limpieza
import pandas as pd
import os

In [17]:
# Asignamos el path relativo a la variable dir para utilizarla a la hora de consumir los datasets
# Obtener el path actual
dir = os.getcwd()

In [18]:
# Cargamos el dataset 'business.parquet' que contiene la lista de negocios
df_yelp_business = pd.read_parquet(dir+'/parquets/business.parquet')


In [19]:
df_yelp_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,None,93101,34.426679,-119.711197,5.0,7.0,0.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,None,63123,38.551126,-90.335695,3.0,15.0,1.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Shipping Centers, Local Services, Notaries, Ma...","{'Friday': '8:0-18:30', 'Monday': '0:0-0:0', '..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,None,85711,32.223236,-110.880452,3.5,22.0,0.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Department Stores, Shopping, Fashion, Home & G...","{'Friday': '8:0-23:0', 'Monday': '8:0-22:0', '..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80.0,1.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Friday': '7:0-21:0', 'Monday': '7:0-20:0', '..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13.0,1.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Brewpubs, Breweries, Food","{'Friday': '12:0-22:0', 'Monday': None, 'Satur..."


In [20]:
df_yelp_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300692 entries, 0 to 300691
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150351 non-null  object 
 1   name          150351 non-null  object 
 2   address       150351 non-null  object 
 3   city          150351 non-null  object 
 4   state         150348 non-null  object 
 5   postal_code   150351 non-null  object 
 6   latitude      150351 non-null  float64
 7   longitude     150351 non-null  float64
 8   stars         150351 non-null  float64
 9   review_count  150351 non-null  float64
 10  is_open       150351 non-null  float64
 11  attributes    136607 non-null  object 
 12  categories    150248 non-null  object 
 13  hours         127128 non-null  object 
dtypes: float64(5), object(9)
memory usage: 32.1+ MB


### Extraemos las categorías del dataset

In [21]:
df = pd.DataFrame()
# Convertir los strings separados por comas en una lista y obtener los valores únicos
df['categories_list'] = df_yelp_business['categories'].str.split(', ')
unique_categories = set(df['categories_list'].explode())
# Convertir el conjunto a una lista
lista_unique_categories = list(unique_categories)

## Creamos una tabla con filtro en la columna 'categories'

In [22]:
df_hotels_yelp = df_yelp_business
# Primero, dividimos los valores de 'categories' en listas de strings
df_hotels_yelp['categories'] = df_hotels_yelp['categories'].str.split(',')

df_hotels_yelp = df_hotels_yelp.dropna(subset=['categories'])

# A continuación, definimos una función que verifique si 'hotels' aparece en la lista,
# y si 'hotels & travel' o 'hotel bar' están ausentes
def filter_categories(categories):
    return 'Hotels' in categories and 'Hotels & Travel' not in categories and 'Hotel Bar' not in categories

# Usamos la función para filtrar las filas y nos quedamos solo con las que cumplen el criterio
df_hotels_yelp = df_hotels_yelp[df_hotels_yelp['categories'].apply(filter_categories)]

# Ahora, si lo deseas, puedes volver a unir los valores de la lista en un solo string separado por comas
df_hotels_yelp['categories'] = df_hotels_yelp['categories'].apply(lambda x: ','.join(x))

# Restablecemos el índice y eliminar la columna "index"
df_hotels_yelp.reset_index(inplace=True, drop=True)

In [23]:
df_hotels_yelp.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,xM6LoUcnpDpMBzXs_7dXAg,Fairfield Inn & Suites,719 E Baltimore Pike,Kennett Square,AB,19348,39.856248,-75.694610,3.0,37.0,1.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Hotels, Hotels & Travel, Event Planning & Serv...",None
1,hUQ9Z7kQeabvhPOAQOVV1A,Rathbone Mansions,1244 Esplanade Ave,New Orleans,IN,70116,29.967055,-90.065828,3.5,67.0,1.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Hotels, Hotels & Travel, Bed & Breakfast, Even...",None
2,xloFoRiYlH4IKGz3FhTDpA,1-275 Rest Area Manatee County Mile 7,13018 Rest Area,Terra Ceia,LA,34250,27.584300,-82.613950,4.0,5.0,1.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Hotels, Rest Stops, Event Planning & Services,...",None
3,XTWGzxXdQN0DVVH2GPGwiA,Days Inn by Wyndham Reno South,"5851 S Virginia St, No.101",Reno,IN,89502,39.472575,-119.788467,2.0,27.0,0.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Hotels, Hotels & Travel, Event Planning & Serv...","{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa..."
4,PXI7kvSV-NvWcqbI9jlmtg,Best Western Cedar Park Inn,5116 Gateway Boulevard,Edmonton,AZ,T6H 2H4,53.488591,-113.493509,3.0,17.0,1.0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Hotels, Event Planning & Services, Hotels & Tr...","{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa..."


In [24]:
df_hotels_yelp.shape

(836, 14)

In [25]:
df_hotels_yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   836 non-null    object 
 1   name          836 non-null    object 
 2   address       836 non-null    object 
 3   city          836 non-null    object 
 4   state         836 non-null    object 
 5   postal_code   836 non-null    object 
 6   latitude      836 non-null    float64
 7   longitude     836 non-null    float64
 8   stars         836 non-null    float64
 9   review_count  836 non-null    float64
 10  is_open       836 non-null    float64
 11  attributes    786 non-null    object 
 12  categories    836 non-null    object 
 13  hours         633 non-null    object 
dtypes: float64(5), object(9)
memory usage: 91.6+ KB


### Filtro el df de hoteles solo en los estados 'CA', 'FL', 'NV', 'NY' y 'TX'

In [26]:
# Filtrar filas donde los valores de la columna 'state' los contanga la siguiente lista: ['CA', 'FL', 'NV', 'NY', 'TX'] 
df_hotels_yelp = df_hotels_yelp[df_hotels_yelp['state'].isin(['CA', 'FL', 'NV', 'NY', 'TX'])]

# Resetear indices
df_hotels_yelp.reset_index(inplace=True,drop=True)


In [27]:
df_hotels_yelp.shape

(220, 14)

In [28]:
df_hotels_yelp.to_csv('hotels_yelp.csv')

### Creo la tabla yelp_hotel_categories que contiene las categorias y sus id

In [29]:
df = pd.DataFrame()
# Convertir los strings separados por comas en una lista y obtener los valores únicos
df['categories_list'] = df_hotels_yelp['categories'].str.split(', ')
unique_hotels_categories = set(df['categories_list'].explode())
# Convertir el conjunto a una lista
lista_hotels_categories = list(unique_hotels_categories)
# Create a DataFrame with columns 'category_id' and 'category'
lista_hotels_categories = pd.DataFrame({'category_id': range(len(lista_hotels_categories)), 'category': lista_hotels_categories})

lista_hotels_categories.reset_index(inplace=True,drop=True)

In [30]:
lista_hotels_categories.to_csv('hotel_categories_yelp.csv', index=False)

In [31]:
# Transformo el dataframe en un diccionario para luego mapear el dataframe 'df_hotels_desanidado'
category_dict = lista_hotels_categories.set_index('category')['category_id'].to_dict()

In [32]:
category_dict

{'Fashion': 0,
 'Airports': 1,
 'Event Planning & Services': 2,
 'Home Services': 3,
 'Hostels': 4,
 "Women's Clothing": 5,
 'Party & Event Planning': 6,
 'Real Estate': 7,
 'Guest Houses': 8,
 'Dance Clubs': 9,
 'Buffets': 10,
 'Apartments': 11,
 'Local Flavor': 12,
 'Beauty & Spas': 13,
 'Tours': 14,
 'Restaurants': 15,
 'American (Traditional)': 16,
 'Music Venues': 17,
 'Hotels & Travel': 18,
 'Resorts': 19,
 'Caterers': 20,
 'Beer Bar': 21,
 'Bed & Breakfast': 22,
 'Casinos': 23,
 'Nightlife': 24,
 'American (New)': 25,
 'Arts & Entertainment': 26,
 'Shopping': 27,
 'Bars': 28,
 'Hotels': 29,
 'Day Spas': 30,
 'Venues & Event Spaces': 31,
 'Massage': 32,
 'Travel Services': 33,
 'Vacation Rentals': 34}

### Creo una tabla que contenga los business_id y los categories_id

In [33]:
#Primero desanido la tabla df_hotels_yelp
df_anidado = df_hotels_yelp.loc[:,['business_id','categories']]

# Dividir los valores separados por comas en filas separadas
df_anidado['categories'] = df_anidado['categories'].str.split(',')

# Usar la función explode para obtener un DataFrame desanidado
df_hotels_desanidado = df_anidado.explode('categories')

#Elimino los espacios en los valores de la columna 'categories'
df_hotels_desanidado['categories'] = df_hotels_desanidado['categories'].str.strip()

# Renombrar la columna 'categories'
df_hotels_desanidado.rename(columns={'categories': 'category'}, inplace=True)

# Elimino los index
df_hotels_desanidado.reset_index(inplace=True, drop=True)

# Mostrar el nuevo DataFrame desanidado
df_hotels_desanidado.head()



,business_id,category
0,kMHZgooGdHN9099Ya0X4vA,Hotels
1,kMHZgooGdHN9099Ya0X4vA,Hotels & Travel
2,kMHZgooGdHN9099Ya0X4vA,Event Planning & Services
3,_d5OAxaxRPsivgYbBq7U1A,Hotels
4,_d5OAxaxRPsivgYbBq7U1A,Event Planning & Services


In [34]:
df_hotels_desanidado.shape

(790, 2)

### Creo una tabla que contenga 'business_id', 'category' y 'category_id'

In [73]:
business_category_yelp = df_hotels_desanidado

# Para agregar la columna 'category_id' al DataFrame, puedes usar el método map:
business_category_yelp['category_id'] = business_category_yelp['category'].map(category_dict)

# Borro la columna 'category'
del business_category_yelp["category"]

business_category_yelp.head()


,business_id,category_id
0,kMHZgooGdHN9099Ya0X4vA,29
1,kMHZgooGdHN9099Ya0X4vA,18
2,kMHZgooGdHN9099Ya0X4vA,2
3,_d5OAxaxRPsivgYbBq7U1A,29
4,_d5OAxaxRPsivgYbBq7U1A,2


In [74]:
business_category_yelp.to_csv('business_category_yelp.csv', index=False)

### Creo una tabla con las columnas 'business_id' del dataframe 'df_hotels_desanidado' y la columna 'category_id' del dataframe 'lista_hotels_categories'

In [37]:

business_category = lista_hotels_categories.merge(df_hotels_desanidado, on="category").sort_values(by="business_id").drop(columns="category").reset_index(drop=True)
business_category.head()

,category_id_x,business_id,category_id_y
0,2,-1ueCbvIpUPi8KT95ETTKw,2
1,29,-1ueCbvIpUPi8KT95ETTKw,29
2,18,-1ueCbvIpUPi8KT95ETTKw,18
3,31,-1ueCbvIpUPi8KT95ETTKw,31
4,18,-7pCvqW8TFs9nBEwlwqNcQ,18


### Creo una tabla que tenga los nombres de los hoteles

In [38]:
df_hotels_name = df_hotels_yelp['name']
df_hotels_name.head()


0    La Quinta Inn by Wyndham Indianapolis East-Pos...
1             Hyatt Place Philadelphia/King of Prussia
2                                          Econo Lodge
3                         Staybridge Suites - Franklin
4                                The Independent Hotel
Name: name, dtype: object

In [39]:
df_hotels_name.to_csv('hotels_name_yelp.csv',index=False)

### Creo tabla de palabras frecuentes en los nombres

In [40]:
# Obtener una Serie con todas las palabras de la columna 'nombre_columna'
palabras = df_hotels_yelp['name'].str.split(expand=True).stack()

# Contar la frecuencia de cada palabra y ordenarlas de mayor a menor frecuencia
frecuencia_palabras = palabras.value_counts().reset_index()
frecuencia_palabras.columns = ['Palabra', 'Frecuencia']
frecuencia_palabras = frecuencia_palabras.sort_values(by='Frecuencia', ascending=False)

print(frecuencia_palabras)

      Palabra  Frecuencia
0         Inn          84
1       Hotel          44
2      Suites          42
3          by          36
4           &          26
..        ...         ...
195      Rice           1
196      Cove           1
197   Oakview           1
198  Microtel           1
343    Indigo           1

[344 rows x 2 columns]


In [41]:
frecuencia_palabras.to_csv('palabras_frecuencia_name_yelp.csv',index=False)

## Trabajamos sobre el dataset checkin

In [42]:
df_yelp_checkin = pd.read_parquet(dir+'/parquets/checkin.parquet')


In [43]:
df_yelp_checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [44]:
# Filtrar filas de df_yelp_checkin donde 'business_id' coincida con los valores de df_hotels_name
yelp_checkin = df_yelp_checkin[df_yelp_checkin['business_id'].isin(df_hotels_yelp['business_id'])]

yelp_checkin.reset_index(inplace=True,drop=True)


In [45]:
yelp_checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  214 non-null    object
 1   date         214 non-null    object
dtypes: object(2)
memory usage: 3.5+ KB


In [46]:
yelp_checkin.to_csv('checkin_yelp.csv',index=False)

# Trabajamos sobre el dataset review

In [47]:
df_yelp_review = pd.read_parquet(dir+'/parquets/review.parquet')


In [48]:
df_yelp_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [49]:
df_yelp_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB


In [50]:
# Creamos una tabla 'review_merge' con las filas filtradas según los valores de la columna 'business_id' de df_final
# Obtener los valores únicos de la columna "business_id" del DataFrame df_final
valores_filtro = df_hotels_yelp['business_id'].unique()

# Filtrar las filas de review_merge basado en los valores de "business_id" que coincidan con "business_id" en df_final
yelp_review = df_yelp_review[df_yelp_review['business_id'].isin(valores_filtro)]

# Restablecemos el índice y eliminar la columna "index"
yelp_review.reset_index(inplace=True, drop=True)

In [51]:
yelp_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,ra9fNjYUumKp_iaqv-jjDg,l_izt6hn7oTxalU06mSyqg,jTI5Xjk27An8ceJ6VwpXiQ,4,0,0,0,"Comfortable bed, good breakfast, fast internet...",2013-08-15 15:57:09
1,Lj1TAURE_hYtPW3IlhKWMg,tWJoKvHynA8h8dFXUExekw,9xm4aIAT_xwRoTxc0_8AWQ,5,0,1,0,Stayed here three nights on business. The pro...,2015-02-16 14:00:30
2,NbT7iIVr8rz6tglE7tYTAg,H4JNrBAoyCk_ZMZWbAf8OA,oEbpO3vmcrGDW2mtb_s8cQ,4,1,0,0,Located right at the Union Station Metro stop ...,2007-05-23 08:03:29
3,gF2MHTRq7XqH_t3XNk9HpA,m3XeAlZN5mBjp7ynqe8_XQ,_d5OAxaxRPsivgYbBq7U1A,5,0,0,0,This hotel was fantastic. I was in town for a ...,2015-07-03 22:33:18
4,lm7Q8pH26kqx1gbjCJ16ZQ,Sh_vUlHHY2Kuj14eF8NYZQ,s1PNBO9o5jIgNd5YWUDLXQ,5,1,0,0,This hotel is a gem in the heart of Philadelph...,2016-02-23 21:31:39


In [52]:
yelp_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10233 entries, 0 to 10232
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   review_id    10233 non-null  object        
 1   user_id      10233 non-null  object        
 2   business_id  10233 non-null  object        
 3   stars        10233 non-null  int64         
 4   useful       10233 non-null  int64         
 5   funny        10233 non-null  int64         
 6   cool         10233 non-null  int64         
 7   text         10233 non-null  object        
 8   date         10233 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 719.6+ KB


In [53]:
yelp_review.to_csv('review_yelp.csv',index=False)

## Trabajamos con el dataset tip

In [54]:
df_yelp_tip = pd.read_parquet(dir+'/parquets/tip.parquet')


In [55]:
df_yelp_tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [56]:
df_yelp_tip.shape

(908915, 5)

In [57]:
# Creamos una tabla 'tip_merge' con las filas filtradas según los valores de la columna 'business_id' de df_final
# Obtener los valores únicos de la columna "business_id" del DataFrame df_final
valores_filtro = df_hotels_yelp['business_id'].unique()

# Filtrar las filas de review_merge basado en los valores de "business_id" que coincidan con "business_id" en df_final
yelp_tip = df_yelp_tip[df_yelp_tip['business_id'].isin(valores_filtro)]

# Restablecemos el índice y eliminar la columna "index"
yelp_tip.reset_index(inplace=True, drop=True)

In [58]:
yelp_tip.head()

,user_id,business_id,text,date,compliment_count
0,TwjiWvK0ZP15lzGC049oIQ,3NdRfVpi3-tfk9vdiJ_F7w,Free wifi in the lobby.,2014-09-21 03:56:10,0
1,FYuSYO1PvJU-I4Hlgio6mA,_d5OAxaxRPsivgYbBq7U1A,The wheelchair accessible rooms are huge!,2012-03-12 22:32:05,0
2,3TeEHdeMOcamovMjZ9d2GA,p6H295aQCbDJ80GcpqFvwA,"Great location, small, but well appointed room...",2011-10-18 06:39:46,0
3,Lrt21UsyyQ2yLJjkSD36kQ,lzo5vrmcxGcsyPMaGrnQVg,Room 803,2016-05-13 17:53:30,0
4,mtmxx_r0fR7XVQJIalLivw,-1ueCbvIpUPi8KT95ETTKw,"Really nice for the price, the rooms are big a...",2014-08-24 16:18:21,0


In [59]:
yelp_tip.shape

(845, 5)

In [60]:
yelp_tip.to_csv('tip_yelp.csv',index=False)

## Trabajamos sobre la tabla user

In [61]:
df_yelp_user = pd.read_parquet(dir+'/user.parquet')

In [62]:
df_yelp_user.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [63]:
df_yelp_user.shape

(2105597, 22)

### Creo una tabla de usuarios que coinciden con las tablas review y otra con tip para futuro posible uso

### Tabla que contiene los usuarios que dejaron reviews a los hoteles del dataset

In [64]:
# Creo tabla 'user_review' que es la tabla 'user' filtrada con los 'user_id' de la tabla 'review_merge'
# Obtener los valores únicos de la columna "business_id" del DataFrame df_final
valores_filtro = yelp_review['user_id'].unique()

# Filtrar las filas de review_merge basado en los valores de "business_id" que coincidan con "business_id" en df_final
yelp_user_review = df_yelp_user[df_yelp_user['user_id'].isin(valores_filtro)]

# Restablecemos el índice y eliminar la columna "index"
yelp_user_review.reset_index(inplace=True, drop=True)

In [65]:
yelp_user_review.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,MGPQVLsODMm9ZtYQW-g_OA,Jelena,1807,2008-01-19 22:50:00,17331,10461,13734,"2008,2009,2010,2011,2012,2013,2014,2015,2016,2...","HpJzEFN1kyuH6VAF_usqKg, F_NG8ZaNVriV_hujoOejIA...",828,...,167,206,395,130,915,3396,1915,1915,696,470
1,OlJ9vcVFB1iEKcZO-MS3cQ,Joz Joz Joz,348,2006-03-03 08:10:12,2063,1260,1537,"2006,2007,2008,2009,2010,2011,2012","iBeN7TtSJrq-dLwj-EevFw, oSN3M4_WKdlTsnpgqPDiBg...",116,...,32,27,47,6,183,543,462,462,87,29
2,KxrKVxdXGkfMJ9XwJZzoLQ,Lisa,950,2008-10-16 21:03:02,1243,368,527,"2010,2011,2012,2013,2014,2015,2016,2017,2018,2...","EWNe5k2pLEefqWnAdC4_1A, C44UVPGmzusO4_a576Wa6g...",35,...,2,1,1,1,26,51,34,34,11,5
3,eqS5lpxVMEzS_xaGP8NW_w,Connie,265,2008-02-14 04:54:46,475,54,261,"2016,2017,2018,2019,20,20,2021","Xkoh-2KxnxL2WYZNbCiy_w, 2EtSs7E4Jl-otU3-XOySIQ...",8,...,3,1,0,0,21,4,30,30,14,0
4,3zxy3LVBV3ttxoYbY4rQ8A,Farrah,2073,2008-05-12 09:13:06,27350,11667,17603,"2008,2009,2010,2011,2012,2013,2014,2015,2016,2...","RtUkSBvInr8mi6-I-PQWoA, rZu5M6e0KtjGJfy_-YqCPQ...",2073,...,180,171,247,162,1575,8734,2651,2651,755,588


In [66]:
yelp_user_review.shape

(12283, 22)

In [67]:
yelp_user_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             12283 non-null  object 
 1   name                12283 non-null  object 
 2   review_count        12283 non-null  int64  
 3   yelping_since       12283 non-null  object 
 4   useful              12283 non-null  int64  
 5   funny               12283 non-null  int64  
 6   cool                12283 non-null  int64  
 7   elite               12283 non-null  object 
 8   friends             12283 non-null  object 
 9   fans                12283 non-null  int64  
 10  average_stars       12283 non-null  float64
 11  compliment_hot      12283 non-null  int64  
 12  compliment_more     12283 non-null  int64  
 13  compliment_profile  12283 non-null  int64  
 14  compliment_cute     12283 non-null  int64  
 15  compliment_list     12283 non-null  int64  
 16  comp

In [68]:
yelp_user_review.to_csv('user_review_yelp.csv',index=False)

### Tabla que contiene los usuarios que dejaron tips a los hoteles del dataset

In [69]:
# Creo tabla 'user_tip' que es la tabla 'user' filtrada con los 'user_id' de la tabla 'tip_merge'
# Obtener los valores únicos de la columna "business_id" del DataFrame df_final
valores_filtro = yelp_tip['user_id'].unique()

# Filtrar las filas de review_merge basado en los valores de "business_id" que coincidan con "business_id" en df_final
yelp_user_tip = df_yelp_user[df_yelp_user['user_id'].isin(valores_filtro)]

# Restablecemos el índice y eliminar la columna "index"
yelp_user_tip.reset_index(inplace=True, drop=True)

In [70]:
yelp_user_tip.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,x4f9anzaaBozvJh5nv26SA,Mandy,309,2010-02-01 14:05:35,832,211,370,"2012,2013,2014,2015,2016","TDnCmO2BGr6PCth0w2ORAQ, oAf9OkoaeE252ijLydHUqA...",50,...,4,0,1,1,18,15,29,29,23,15
1,4Pwav7HENZumUr3RJE03ig,Walter,26,2010-12-19 02:46:38,40,12,9,,"I7RhGhBlO5WPGevirQttRA, M45pzaNPMVXkkxmuJ0p_og...",1,...,0,0,0,0,0,1,1,1,1,0
2,1sGYXSkJHPhJ6wQtc-RbZw,Kimberly,820,2009-07-12 01:03:14,5441,2976,4264,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","9w4vI3LRTA4JdJyzOxeEMw, fNSGyM1NJ91CTO_VfJ5asQ...",149,...,31,43,12,8,154,403,541,541,265,148
3,0iq45WV_h_j8SXjR4ytcJg,A,186,2009-12-08 23:21:45,669,301,379,,"dXpgRqVIJZZcD87Tf9DtUQ, 5pDJuri4g3Wfes8GGlnudA...",46,...,2,1,0,0,7,22,18,18,4,4
4,u1cvQWB0lB4YsIbtSlUR7g,Barb,150,2009-11-03 04:56:21,150,32,73,"2013,2014,2015,2016,2017","GGvBWumvNzl8i54cKlACmQ, 1xJt3jMREtMxL0zWEaWryA...",11,...,2,3,0,0,5,22,19,19,4,4


In [71]:
yelp_user_tip.shape

(1138, 22)

In [72]:
yelp_user_tip.to_csv('user_tip_yelp.csv',index=False)